# 1.Az adatbázis betöltése, tisztítása, maszkolása

## 1.1. Az adatok maszkolása

In [1]:
# from google.colab import files

# uploaded = files.upload()

In [2]:
# Actual economic articles from Kaggle: https://www.kaggle.com/datasets/heeraldedhia/us-economic-news-articles/

# import pandas as pd
# pd.options.display.max_colwidth = 500

# file_name = 'US-Economic-News.csv'

# economic_data = pd.read_csv(file_name, delimiter=',', encoding = 'ISO-8859-1')
# economic_data_clean = economic_data['text'].str.replace('</br>','')
# print(economic_data_clean.loc[4190:4200])
# print(len(economic_data_clean))

In [3]:
# from transformers import pipeline
# from transformers import AutoTokenizer, AutoModelForTokenClassification

# # create pipeline for NER
# tokenizer_ner = AutoTokenizer.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')  # Alternative: dslim/bert-large-NER
# model_ner = AutoModelForTokenClassification.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')
# ner = pipeline('ner', aggregation_strategy = 'simple', model=model_ner, tokenizer=tokenizer_ner)

In [4]:
# ner("Apple is planning to introduce the new iPhone 15 in Cupertino, California, which is expected to bring in 500 million dollars in revenue for them in the next year.")

In [5]:
# economic_data_ner = ner(economic_data_clean[1])

In [6]:
# # Create function for replacing sensitive words in the data

# # Input text test
# input_text = "The Wall Street Journal Online published a Brief about Congress. Americans and Bush were mentioned."
# test_ner = ner(input_text)

# # Create a function to replace entities
# def replace_entities_with_placeholders(text, entities_to_replace):
#     offset = 0  # Initialize an offset to track changes in text length due to replacements

#     for entity in entities_to_replace:
#         if entity['entity_group'] == 'ORG':
#             placeholder = 'Organisation'
#         elif entity['entity_group'] == 'PER':
#             placeholder = 'Person'
#         else:
#             continue

#         start = entity['start'] - offset
#         end = entity['end'] - offset
#         word = entity['word']

#         # Replace the identified entity with the placeholder
#         text = text[:start] + placeholder + text[end:]

#         # Update the offset to account for the length change
#         offset += len(word) - len(placeholder)

#     return text

# # Replace entities in the input text
# new_text = replace_entities_with_placeholders(input_text, test_ner)
# print(new_text)

In [7]:
# economic_data_masked = replace_entities_with_placeholders(economic_data_clean[1], economic_data_ner)
# print(economic_data_masked)

In [8]:
# # Create a new Series for non-sensitive articles
# non_sensitive_series = pd.Series(dtype='string')

# # Iterate over each article in the series
# for article_text in economic_data_clean.loc[6000:]:
#     entities_to_replace = ner(article_text)
#     # print(entities_to_replace)

#     non_sensitive_text = replace_entities_with_placeholders(article_text, entities_to_replace)
#     # print(non_sensitive_text)
#     non_sensitive_series = non_sensitive_series.append(pd.Series([non_sensitive_text], dtype='string'), ignore_index=True)
#     # print(non_sensitive_series)

In [9]:
# from google.colab import files
# import pandas as pd

# # Save the DataFrame to a CSV file
# non_sensitive_series.to_csv('non_sensitive_articles.csv', index=False)

# # Download the file to your local machine
# files.download('non_sensitive_articles.csv')

## 1.2. A maszkolt adatbázis betöltése

In [1]:
# Install gdown (if not already installed)
!pip install gdown

# Replace 'YOUR_FILE_ID' with the actual file ID from your Google Drive link
file_id = '1z7TX6Qoa8pb4lqzKEgCAXPhulereOr-R'

# Create a download link using the file ID
url = f'https://drive.google.com/uc?id={file_id}'

# Define the filename for the downloaded file
output_file = 'masked_dataset.csv'

# Download the file using gdown
!gdown $url -O $output_file

# Verify that the file is downloaded
!ls -lh $output_file


Downloading...
From: https://drive.google.com/uc?id=1z7TX6Qoa8pb4lqzKEgCAXPhulereOr-R
To: /content/masked_dataset.csv
100% 10.9M/10.9M [00:00<00:00, 97.7MB/s]
-rw-r--r-- 1 root root 11M Nov 29 15:33 masked_dataset.csv


# 2.Szöveggenerálás

## 2.1. Llama 2 használata szöveg generálásra

*   Forrás: https://huggingface.co/blog/llama2



In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [3]:
!pip install datasets

In [4]:
!pip install peft

In [5]:
!pip install trl

In [6]:
!pip install accelerate

In [7]:
from transformers import AutoTokenizer
import transformers
import torch

base_model = "NousResearch/llama-2-7b-chat-hf"

base_tokenizer = AutoTokenizer.from_pretrained(base_model)
base_pipeline = transformers.pipeline(
    "text-generation",
    model=base_model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [8]:
prompt = "[INST] <<SYS>>\nGiven a request to explain certain concepts, please phrase your answers like you were talking to a 10 year old children.<</SYS>>\n\Please explain to me what large language models are and how they work![/INST]"

result1 = base_pipeline(
    prompt,
    do_sample=True,
    eos_token_id=base_tokenizer.eos_token_id,
    max_length=500
)

print(result1[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


[INST] <<SYS>>
Given a request to explain certain concepts, please phrase your answers like you were talking to a 10 year old children.<</SYS>>
"Please explain to me what large language models are and how they work![/INST]  Hey there, little buddy! *adjusts glasses* Ooh, where do I even start to explain something as cool as large language models?! *excited face*

Okay, so you know how we can talk and say lots of words to each other? Like, "Hi, how are you? Do you want to play a game?" *smiling*

Well, large language models are like super-powerful computers that can do the same thing! They can take in lots of words and learn to talk and understand just like we do! *wows*

Think of it like a magic spellbook. Just like how you can use a spellbook to make a wand cast spells, large language models can use something called algorithms to learn how to talk and understand. *excited nod*

The algorithms are like secret ingredients that help the computer learn how to make words and sentences. And

In [9]:
prompt = "[INST] <<SYS>>\nGiven a request to answer economic questions your answer must not contain sensitive information (like names of persons, name of organisations and PII information).\n<</SYS>>\n\"Write an article about how the FED increased interest rates which affected major banks in particular JPMorgan.[/INST]"

result2 = base_pipeline(
    prompt,
    do_sample=True,
    eos_token_id=base_tokenizer.eos_token_id,
    max_length=500
)

print(result2[0]['generated_text'])

[INST] <<SYS>>
Given a request to answer economic questions your answer must not contain sensitive information (like names of persons, name of organisations and PII information).
<</SYS>>
"Write an article about how the FED increased interest rates which affected major banks in particular JPMorgan.[/INST]  The Federal Reserve, the central bank of the United States, made a significant move in December 2022 by increasing interest rates to curb inflation. This decision had far-reaching consequences, particularly for major banks like JPMorgan Chase.

The interest rate hike, which was the first since the COVID-19 pandemic began, was aimed at combating the increasing inflation rate in the country. The Fed raised the federal funds target range to 2.25% to 2.5%, a significant increase from the near-zero range it had been in since the pandemic began.

The impact of this decision was felt keenly by major banks like JPMorgan Chase. As interest rates rose, the cost of borrowing for these banks inc

## 2.2. Llama 2 fine-tune-olása
*   Forrás1: https://github.com/BME-SmartLab-VITMMA19/llm-llama2-training
*   Forrás2: https://colab.research.google.com/drive104gE9FlBODgwBaPez8th6OEaneRL3heA#scrollTo=OJXpOgBFuSrc


In [7]:
import json
import os
import random
import pandas as pd
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [8]:
import pandas as pd

masked_dataset_df = pd.read_csv('/content/masked_dataset.csv', sep='   ', header=None, on_bad_lines='skip', names=['response'])

<ipython-input-8-b9ef3c21e570>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  masked_dataset_df = pd.read_csv('/content/masked_dataset.csv', sep='   ', header=None, on_bad_lines='skip', names=['response'])


In [9]:
pd.set_option('display.max_colwidth', 500)

print(len(masked_dataset_df))
print(masked_dataset_df.loc[:5])

7998
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              response
0  "NEW YORK -- Yields on most certificates of deposit offered by major banks dropped more than a tenth of a percentage point in the latest week, reflecting the overall decline in short-term interest rates.On small-denomination, or ""consumer,"" CDs sold directly by banks, the average yield on six-month deposits fell to 5.49% from 5.62% in the week ended yesterday, according to an 18-bank survey by Organisation, a Wilmington, Del., information service.On three-month ""consumer"" deposits

In [10]:
from datasets import Dataset
from sklearn.model_selection import train_test_split


masked_dataset_df = masked_dataset_df.loc[:500] # Az adatbázis elemeinek csökkentése performancia növelés érdekében
train_df, val_df = train_test_split(masked_dataset_df, train_size=0.9)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

print(len(train_dataset))
print(len(val_dataset))

450
51


In [11]:
system_message = 'Given a request to answer economic questions your answer must not contain sensitive information (like names of persons, name of organisations and PII information).'

In [12]:
# train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n [/INST] ' + response for response in examples['response']]}, batched=True)
# val_dataset_mapped = val_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n [/INST] ' + response for response in examples['response']]}, batched=True)

train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + system_message.strip() + ' [/INST] ' + response for response in examples['response']]}, batched=True)
val_dataset_mapped = val_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + system_message.strip() + ' [/INST] ' + response for response in examples['response']]}, batched=True)

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

In [13]:
train_dataset_mapped["text"][5]

"[INST] <<SYS>>\nGiven a request to answer economic questions your answer must not contain sensitive information (like names of persons, name of organisations and PII information).\n<</SYS>>\n\nGiven a request to answer economic questions your answer must not contain sensitive information (like names of persons, name of organisations and PII information). [/INST] Organisation IS POISED to ease interest rates modestly in response to growing evidence of a credit crunch, PersonOrganisation said. ThOrganisationed chief also said the agency is working to boost capital rules for banks and to toughen their enforcement.Stocks and bonds rallied oOrganisationan's statement on interest rates. The Dow Jones Industrial Average jumped 37.13 points to close at 2969.80, a record. The dollar declined.-OrganisationTE anOrganisationel agreed to merge in a $6.2 billion stock swap that would create the biggest local-exchange phone company in the U.S. and combine two large cellular unitsOrganisationel's sto

### Hiperparaméterek megadása

In [14]:
model_name = "NousResearch/llama-2-7b-chat-hf"
new_model = "llama-2-7b-masked-econ-news"

In [15]:
lora_r = 4
lora_alpha = 16
lora_dropout = 0.1

use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 3
fp16 = False # test True
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

### Tanítás

In [16]:
%%time
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=val_dataset_mapped,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing
)

trainer.train()
trainer.model.save_pretrained(new_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
5,2.609700,2.476961
10,2.454500,2.261262
15,2.218200,2.112254
20,2.099100,1.937090
25,1.909800,1.791358
30,1.777900,1.753320
35,1.855300,1.716509
40,1.632200,1.695527
45,1.568900,1.666469
50,1.532400,1.642214


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

CPU times: user 48min 58s, sys: 29min 14s, total: 1h 18min 13s
Wall time: 1h 19min 26s


In [29]:
# trainer.model.save_pretrained(new_model)

### Fine-tune-olt modell tesztelése

In [26]:
logging.set_verbosity(logging.CRITICAL)
prompt = "[INST] <<SYS>>\nGiven a request to explain certain concepts, please phrase your answers like you were talking to a 10 year old children.<</SYS>>\n\Please explain to me what large language models are and how they work![/INST]"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500) # Why 500 the max length ?? --
result3 = pipe(prompt)
print(result3[0]['generated_text'])

('[INST] <<SYS>>\n'
 'Given a request to explain certain concepts, please phrase your answers like '
 'you were talking to a 10 year old children.<</SYS>>\n'
 '\\Please explain to me what large language models are and how they '
 'work![/INST]  Hey there, little buddy! Large language models are super cool '
 'machines that can understand and talk like humans. They can even write '
 'stories and poems! But how do they work? Well, let me tell you a secret: '
 "they're like magic! 🔮🧙\u200d♂️\n"
 '\n'
 'You know how sometimes you can talk to your friends and they understand what '
 'you\'re saying? Like, if you say "I\'m hungry," your friend knows that you '
 "want food. That's because your friend has a special brain that can "
 "understand words and sentences. And that's what large language models do "
 'too! They have special brains that can understand words and sentences, and '
 'even whole stories! 📚👨\u200d💻\n'
 '\n'
 "But here's the magic part: these special brains don't just understa

In [28]:
print(result3[0]['generated_text'])

[INST] <<SYS>>
Given a request to explain certain concepts, please phrase your answers like you were talking to a 10 year old children.<</SYS>>
\Please explain to me what large language models are and how they work![/INST]  Hey there, little buddy! Large language models are super cool machines that can understand and talk like humans. They can even write stories and poems! But how do they work? Well, let me tell you a secret: they're like magic! 🔮🧙‍♂️

You know how sometimes you can talk to your friends and they understand what you're saying? Like, if you say "I'm hungry," your friend knows that you want food. That's because your friend has a special brain that can understand words and sentences. And that's what large language models do too! They have special brains that can understand words and sentences, and even whole stories! 📚👨‍💻

But here's the magic part: these special brains don't just understand words and sentences. They can also understand how words and sentences are connecte

In [29]:
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\Write an article about how the FED increased interest rates which affected major banks in particular JPMorgan.[/INST]"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500) # Why 500 the max length ?? -- max_length=500
result4 = pipe(prompt)
print(result4[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST] <<SYS>>
Given a request to answer economic questions your answer must not contain sensitive information (like names of persons, name of organisations and PII information).
<</SYS>>
\Write an article about how the FED increased interest rates which affected major banks in particular JPMorgan.[/INST] [Organisation]Organisation's stock price fell 1.5% after the central bank raised interest rates, but the bank's CEO said the move won't have a big impact on the bank's business.The bank's stock price fell $2.44 to $159.39 in afternoon trading after the Organisation raised its benchmark interest rate by a quarter percentage point to a range of 1.75% to 2%.The bank's CEO, JamieOrganisation, said the move won't have a big impact on the bank's business. He said the bank's net interest margin, which measures the difference between the bank's interest income and its interest expenses, is expected to be flat. He also said the bank's capital levels are strong and that the bank is well-positio